In [13]:
# activate the venv and pip install first
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

print(os.getenv("PINECONE_API_KEY"))

84f7936f-fe8b-4a5b-8fe8-5bbb07bc8ecb


In [14]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [ ]:
# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [3]:
# Load the review data
data = json.load(open("reviews.json"))

processed_data = []
client = OpenAI()

data['reviews']

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [4]:
# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

NameError: name 'client' is not defined

In [5]:
processed_data[0]

IndexError: list index out of range

In [ ]:
index = pc.Index('rag')
index.upsert 